In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')
print(df.head())

    lead_source    industry  number_of_courses_viewed  annual_income  \
0      paid_ads         NaN                         1        79450.0   
1  social_media      retail                         1        46992.0   
2        events  healthcare                         5        78796.0   
3      paid_ads      retail                         2        83843.0   
4      referral   education                         3        85012.0   

  employment_status       location  interaction_count  lead_score  converted  
0        unemployed  south_america                  4        0.94          1  
1          employed  south_america                  1        0.80          0  
2        unemployed      australia                  3        0.69          1  
3               NaN      australia                  1        0.87          0  
4     self_employed         europe                  3        0.62          1  


In [2]:
df['industry'].mode([0])

0    retail
Name: industry, dtype: object

In [3]:
#data preparation
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [4]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [5]:
df.lead_source = df.lead_source.fillna('NA')
df.industry = df.industry.fillna('NA')
df.annual_income = df.annual_income.fillna(0)
df.employment_status = df.employment_status.fillna('NA')
df.location = df.location.fillna('NA')
print(df.isnull().sum())

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64


In [6]:
df['industry'].mode([0])


0    retail
Name: industry, dtype: object

In [10]:
correlation = df['interaction_count'].corr(df['lead_score'])
print(correlation)
correlation1 = df['number_of_courses_viewed'].corr(df['lead_score'])
print(correlation1)
correlation2 = df['number_of_courses_viewed'].corr(df['interaction_count'])
print(correlation2)
correlation3 = df['annual_income'].corr(df['interaction_count'])
print(correlation3)

0.0098881824969131
-0.004878998354681263
-0.023565222882888103
0.027036472404814327


In [17]:
#Q3 splitting the data
from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
len(df_full_train), len(df_test)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)
len(df_train), len(df_val), len(df_test)


(876, 293, 293)

In [18]:
#drop indexes
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

print(df_train.head())

      lead_source       industry  number_of_courses_viewed  annual_income  \
0        paid_ads         retail                         0        58472.0   
1  organic_search  manufacturing                         3        71738.0   
2        paid_ads     technology                         3        81973.0   
3              NA     technology                         1        74956.0   
4  organic_search         retail                         3        59335.0   

  employment_status       location  interaction_count  lead_score  converted  
0           student    middle_east                  5        0.03          0  
1           student    middle_east                  6        0.77          1  
2          employed  north_america                  2        0.59          1  
3          employed         europe                  3        0.34          1  
4           student      australia                  1        0.98          1  


In [20]:
# preparing y
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values
del df_train['converted']
del df_val['converted']
del df_test['converted']

print(df_train, y_train)

        lead_source       industry  number_of_courses_viewed  annual_income  \
0          paid_ads         retail                         0        58472.0   
1    organic_search  manufacturing                         3        71738.0   
2          paid_ads     technology                         3        81973.0   
3                NA     technology                         1        74956.0   
4    organic_search         retail                         3        59335.0   
..              ...            ...                       ...            ...   
871  organic_search          other                         1        43907.0   
872    social_media         retail                         3        64969.0   
873              NA      education                         3        89042.0   
874    social_media  manufacturing                         1            0.0   
875          events        finance                         4        50259.0   

    employment_status       location  interaction_c